In [1]:
import csv, random, numpy as np
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img, flip_axis, random_shift

from skimage import exposure
import itertools

Using TensorFlow backend.


In [2]:
def model(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

    conv_layers, dense_layers = [32, 32, 64, 128], [1024, 512]
    
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, activation='elu', input_shape=shape))
    model.add(MaxPooling2D())
    for cl in conv_layers:
        model.add(Convolution2D(cl, 3, 3, activation='elu'))
        model.add(MaxPooling2D())
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl, activation='elu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model

In [3]:
def get_X_y(data_file, flag='unix'):
    """Read the log file and turn it into X/y pairs. Add an offset to left images, remove from right images."""
    X, y = [], []
#    images = []
    steering_offset = 0.4
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):
            
            if(flag == 'window'):
                for copy in range(5):
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]

            left_img = left_img.split('/')[-1]
            right_img = right_img.split('/')[-1]
            center_img = center_img.split('/')[-1]
            
            if float(speed) < 3: continue  # throw away low-speed samples
            X += [left_img.strip(), right_img.strip(),center_img.strip()]
            y += [float(steering_angle) + steering_offset, float(steering_angle) - steering_offset,float(steering_angle)]
    return X, y

In [4]:
def random_darken(image):
    """Given an image (from Image.open), randomly darken a part of it."""
    w, h = image.size

    # Make a random box.
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(x1, w), random.randint(y1, h)

    # Loop through every pixel of our box (*GASP*) and darken.
    for i in range(x1, x2):
        for j in range(y1, y2):
            new_value = tuple([int(x * 0.5) for x in image.getpixel((i, j))])
            image.putpixel((i, j), new_value)
    return image

In [5]:
def convert_gray_with_adapthist_debug(X, y = None):

    #Convert to grayscale, e.g. single Y channel
    X = 0.299 * X[:, :, 0] + 0.587 * X[:, :, 1] + 0.114 * X[:, :, 2]
    #Scale features to be in [0, 1]
    
    X = (X / 255.).astype(np.float32)
    
 
    exposure.equalize_adapthist(X)
    # Apply localized histogram localization  
#    for i in range(X.shape[0]):
#        X[i] = exposure.equalize_adapthist(X[i])
        
#    print("y is %s" % y)
        
    if y is not None:  
        # Convert to one-hot encoding. Convert back with `y = y.nonzero()[1]`
        y = np.eye(43)[y]
        # Shuffle the data
        X, y = shuffle(X, y)

    # Add a single grayscale channel
#    X = X.reshape(X.shape + (1,)) 
    
    return X, y

def process_image(path, steering_angle, augment, shape=(220,320)):
    """Process and augment an image."""
    
    path = '../data2/IMG/' + path 
    
    image = load_img(path, target_size=shape)
    
    if augment and random.random() < 0.5:
        image = random_darken(image)  # before numpy'd

    image = img_to_array(image)
        
    if augment:
        image = random_shift(image, 0, 0.2, 0, 1, 2)  # only vertical
        if random.random() < 0.5:
            image = flip_axis(image, 1)
            steering_angle = -steering_angle

    image = (image / 255. - .5).astype(np.float32)
    
    print(image.shape)
    
    return image, steering_angle

In [6]:
def process_image(path, steering_angle, augment, shape=(220,320)):
    """Process and augment an image."""
    
#    path = '../data2/' + path 
#    path = '../data2/IMG/' + path 
    path = '../merge_image/image/' + path 
    
    #    path = '../data2/' + path 
    
    image = load_img(path, target_size=shape)
    
    if augment and random.random() < 0.5:
        image = random_darken(image)  # before numpy'd

    image = img_to_array(image)
#    print(image)
    
#    print(image.shape)
        
    if augment:
        image = random_shift(image, 0, 0.2, 0, 1, 2)  # only vertical
        if random.random() < 0.5:
            image = flip_axis(image, 1)
            steering_angle = -steering_angle

#    image = convert_gray_with_adapthist_debug(image)
    
#    image = (image[0] - .5).astype(np.float32)

    image = (image / 255. - .5).astype(np.float32)
    
    image = image[50:190, 0:320] 
    
#    image = image.reshape(image.shape + (1,)) 
    
#    print(type(image))
#    print(image.shape)
    
    return image, steering_angle

In [7]:
def _generator(batch_size, X, y):
    """Generate batches of training data forever."""
    while 1:
        batch_X, batch_y = [], []
        for i in range(batch_size):
            sample_index = random.randint(0, len(X) - 1)
            sa = y[sample_index]
            image, sa = process_image(X[sample_index], sa, augment=True)
            batch_X.append(image)
            batch_y.append(sa)
        yield np.array(batch_X), np.array(batch_y)

In [8]:
def train():
    """Load our network and our data, fit the model, save it."""
    net = model(load=False, shape=(140, 320, 3))
    X, y = get_X_y('../data2/driving_log.csv')
    print(type(X))
    
    X2, y2 = get_X_y('../20170320/driving_log.csv', flag='window')
    
    print(type(X2))
    print(X2[0])
    print(len(X2))
    
    X = X + X2
    y = y + y2
    
    print(type(X))
    print(X[0])

    
    net.fit_generator(_generator(128, X, y), samples_per_epoch=20224, nb_epoch=2)
    net.save('short.h5')

In [9]:
train()

<class 'list'>
<class 'list'>
left_2017_03_20_20_02_42_608.jpg
591
<class 'list'>
left_2016_12_01_13_30_48_287.jpg
Epoch 1/2
20224/20224 [==============================] - 269s - loss: 0.1250     